<a href="https://colab.research.google.com/github/Wamp1re-Ai/Google-Colab_Notebooks/blob/main/ZonosTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text-to-Speech with Zonos



In [3]:
# %% CELL 1: Environment Setup and Installation
!apt update && apt install -y espeak-ng
!git clone https://github.com/Isi-dev/Zonos.git
%cd Zonos

# CRITICAL FIX 1: Install the REQUIRED numpy version FIRST.
print("Installing required numpy version (1.26.4)...")
# Use --no-cache-dir to potentially avoid issues with cached wheels
!pip install numpy==1.26.4 --force-reinstall --no-cache-dir

# CRITICAL FIX 2: Install zonos AFTER numpy is set.
# Pip should now respect the existing numpy 1.26.4 installation.
print("Installing Zonos library...")
# Use --no-cache-dir here too for consistency
!pip install -e . --no-cache-dir

print("\n*** Installation Complete. Please RESTART THE RUNTIME now! ***")
print("*** (Runtime -> Restart Runtime) or (Runtime -> Restart Session) ***")
print("*** After restarting, run the next cell (Cell 2). ***")

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
43 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR


*** Installation Complete. Please RESTART THE RUNTIME now! ***
*** (Runtime -> Restart Runtime) or (Runtime -> Restart Session) ***
*** After restarting, run the next cell (Cell 2). ***


In [1]:
# %% CELL 2: Load Model and Run Code (Run AFTER restarting runtime)
import sys
import os

# Ensure we are in the right directory after restart
if os.getcwd() != '/content/Zonos':
  try:
    os.chdir('/content/Zonos')
    print("Changed directory back to /content/Zonos")
  except FileNotFoundError:
    print("ERROR: /content/Zonos directory not found. Please re-run Cell 1.")
    raise

# Add Zonos directory to path if needed (less critical after restart, but safe)
if '/content/Zonos' not in sys.path:
    sys.path.insert(0, '/content/Zonos')
    print("Added /content/Zonos to sys.path")


import torch
import torchaudio

# Verify numpy version *after restart*
import numpy
print(f"Using numpy version: {numpy.__version__}") # SHOULD print 1.26.4 now

# Now try the imports
print("Importing Zonos and related libraries...")
try:
    from zonos.model import Zonos
    from zonos.conditioning import make_cond_dict
    import transformers # Explicitly import transformers to verify
    print("Imports successful!")
except ImportError as e:
    print(f"Import failed: {e}")
    print("ERROR: Imports failed even after restart. Dependency conflict persists.")
    print("Check the Zonos repository issues or try a different environment.")
    raise e
except Exception as e:
    print(f"An unexpected error occurred during import: {e}")
    raise e

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load model
print("Loading model...")
model = Zonos.from_pretrained("Isi99999/Zonos-v0.1-transformer", device=device)
print("Model loaded!")

# %% Optional: Add code here to use the model
# Example:
# text = "Hello world, this is a test."
# lang = "en"
# # Prepare conditioning dictionary
# cond = make_cond_dict(text=text, lang=lang)
# # Generate audio
# print("Generating audio...")
# wav = model.generate(cond=cond, length=None) # Let model determine length
# # Save or play the audio
# torchaudio.save("output.wav", wav.cpu(), model.sr)
# print("Audio saved to output.wav")

Changed directory back to /content/Zonos
Added /content/Zonos to sys.path
Using numpy version: 1.26.4
Importing Zonos and related libraries...
Imports successful!
Using device: cpu
Loading model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

Model loaded!


## Upload Reference Audio (Optional)

In [2]:
# %% [markdown]
# **UPLOAD 10 TO 30 SECONDS REFERENCE VOICE AUDIO** (optional)
# %%
# @title
import os
from google.colab import files

os.environ["LC_ALL"] = "C.UTF-8"
os.environ["LANG"] = "C.UTF-8"
os.makedirs("assets", exist_ok=True)

uploaded = files.upload()
for filename in uploaded.keys():
    new_path = "assets/reference.mp3"
    if os.path.exists(new_path):
        os.remove(new_path)
    os.rename(filename, new_path)  # Rename safely

print("Loading reference audio...")
wav, sampling_rate = torchaudio.load("assets/reference.mp3")
speaker = model.make_speaker_embedding(wav, sampling_rate)
print("Reference audio loaded!")

Loading reference audio...


RuntimeError: Failed to open the input "assets/reference.mp3" (No such file or directory).
Exception raised from get_input_format_context at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/stream_reader/stream_reader.cpp:42 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x96 (0x78a33016c1b6 in /usr/local/lib/python3.11/dist-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x78a330115a76 in /usr/local/lib/python3.11/dist-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42034 (0x78a330069034 in /usr/local/lib/python3.11/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: torio::io::StreamingMediaDecoder::StreamingMediaDecoder(std::string const&, std::optional<std::string> const&, std::optional<std::map<std::string, std::string, std::less<std::string>, std::allocator<std::pair<std::string const, std::string> > > > const&) + 0x14 (0x78a33006b

## Enter Text, Adjust Settings & Run

In [3]:
# %% [markdown]
# **ENTER TEXT, ADJUST SETTINGS & RUN**
# %%
import torch # Importing torch here ensures it's available

text = " I am motivated by the simple yet profound joys of being alive—the taste of a good meal, the laughter of a friend, the beauty of a sunrise, and the endless pursuit of knowledge. Even if everything about me ceases when I die, my actions, words, and ideas can leave ripples in the world, affecting others in ways I may never fully grasp. " # @param {type:"string"}
seed = 421 # @param {"type":"number"}
use_default_speaker = True  # @param {type:"boolean"}
language = 'en-us' # @param ['af', 'am', 'an', 'ar', 'as', 'az', 'ba', 'bg', 'bn', 'bpy', 'bs', 'ca', 'cmn', 'cs', 'cy', 'da', 'de', 'el', 'en-029', 'en-gb', 'en-gb-scotland', 'en-gb-x-gbclan', 'en-gb-x-gbcwmd', 'en-gb-x-rp', 'en-us', 'eo', 'es', 'es-419', 'et', 'eu', 'fa', 'fa-latn', 'fi', 'fr-be', 'fr-ch', 'fr-fr', 'ga', 'gd', 'gn', 'grc', 'gu', 'hak', 'hi', 'hr', 'ht', 'hu', 'hy', 'hyw', 'ia', 'id', 'is', 'it', 'ja', 'jbo', 'ka', 'kk', 'kl', 'kn', 'ko', 'kok', 'ku', 'ky', 'la', 'lfn', 'lt', 'lv', 'mi', 'mk', 'ml', 'mr', 'ms', 'mt', 'my', 'nb', 'nci', 'ne', 'nl', 'om', 'or', 'pa', 'pap', 'pl', 'pt', 'pt-br', 'py', 'quc', 'ro', 'ru', 'ru-lv', 'sd', 'shn', 'si', 'sk', 'sl', 'sq', 'sr', 'sv', 'sw', 'ta', 'te', 'tn', 'tr', 'tt', 'ur', 'uz', 'vi', 'vi-vn-x-central', 'vi-vn-x-south', 'yue']
happy = 0.3077 # @param {type:"slider", min:0.0, max:1.0, step:0.05}
sad = 0.0256 # @param {type:"slider", min:0.0, max:1.0, step:0.05}
disgust = 0.0256 # @param {type:"slider", min:0.0, max:1.0, step:0.05}
fear = 0.0256 # @param {type:"slider", min:0.0, max:1.0, step:0.05}
surprise = 0.0256 # @param {type:"slider", min:0.0, max:1.0, step:0.05}
anger = 0.0256 # @param {type:"slider", min:0.0, max:1.0, step:0.05}
other = 0.2564 # @param {type:"slider", min:0.0, max:1.0, step:0.05}
neutral = 0.3077 # @param {type:"slider", min:0.0, max:1.0, step:0.05}
pitch = 20 # @param {type:"slider", min:0, max:400, step:1}
speed = 15 # @param {type:"slider", min:0.0, max:40.0, step:1.0}

# Define device here to make it accessible
device = "cuda" if torch.cuda.is_available() else "cpu"  # Define device
print(f"Using device: {device}")


total = happy + sad + disgust + fear + surprise + anger + other + neutral
if total > 0:
    happy = happy / total
    sad = sad / total
    disgust = disgust / total
    fear = fear / total
    surprise = surprise / total
    anger = anger / total
    other = other / total
    neutral = neutral / total

emotions = torch.tensor(list(map(float, [happy, sad, disgust, fear, surprise, anger, other, neutral])), device=device)

if use_default_speaker:
    print("Loading default audio...")
    wav, sampling_rate = torchaudio.load("assets/exampleaudio.mp3")
    speaker = model.make_speaker_embedding(wav, sampling_rate)
    print("Default audio loaded!")


def generate_speech2( text, seed = 421, language="en-us", emotion_tensor= torch.tensor(list(map(float, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])), device=device), pitch= 20, speed= 15):
    """Generate speech from text"""
    print(f"Generating: {text}")

    if seed >= 0:
            torch.manual_seed(seed)
    else:
        torch.random.seed()

    # Create conditioning
    cond_dict = make_cond_dict(
        text=text,
        language=language,
        speaker=speaker,
        emotion=emotion_tensor,
        pitch_std = pitch,
        speaking_rate=speed

    )
    conditioning = model.prepare_conditioning(cond_dict)

    # Generate audio
    codes = model.generate(conditioning)
    wavs = model.autoencoder.decode(codes).cpu()

    # Save and play
    filename = "output.wav"
    torchaudio.save(filename, wavs[0], model.autoencoder.sampling_rate)
    return filename

output_file = generate_speech2(text, seed = seed, language=language, emotion_tensor= emotions, pitch = pitch, speed = speed)
from IPython.display import Audio
Audio(output_file)

Using device: cpu
Loading default audio...


ResNet293_SimAM_ASP_base.pt:   0%|          | 0.00/397M [00:00<?, ?B/s]

ResNet293_SimAM_ASP_base_LDA-128.pt:   0%|          | 0.00/265k [00:00<?, ?B/s]

Default audio loaded!
Generating:  I am motivated by the simple yet profound joys of being alive—the taste of a good meal, the laughter of a friend, the beauty of a sunrise, and the endless pursuit of knowledge. Even if everything about me ceases when I die, my actions, words, and ideas can leave ripples in the world, affecting others in ways I may never fully grasp. 


Generating:   0%|          | 0/2588 [00:00<?, ?it/s]No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


KeyboardInterrupt: 